# Project Voyager
#### Author: Jesse Declercq

In [1]:
#-----------------------------------Importing Dependencies and Starting Serial Port---------------------------------------
from IPython.display import clear_output
import numpy as np
import keyboard
import serial
import struct
import math
import time
import sys
serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)

In [2]:
#---------------------------------------------Inverse Kinematics Solver ----------------------------------------------------
def Inverse_Kinematics(X,Y,Z,phi):
    phi = np.radians(phi)
    l2 = 0.130
    l3 = 0.130
    l4 = 0.200 # This parameter changes the end effector location during inverse kinematics
    d = 0.059
    Theta2_list = []
    Theta3_list = []
    error_net = []
    xo = []
    yo = []
    zo = []
    actuallength = (X**2 + Y**2 + Z**2)**0.5 
    MAX = l2 + l3 + l4
    if MAX < actuallength:
        print('The following request is impossible to satisfy')
        sys.exit()  
    Theta1 = np.arctan2(Z,X)
    for i in np.arange(30,151,1):
        Theta2 = np.radians(i)  
        C3 = X/np.cos(Theta1) - l4*np.cos(phi) - l2*np.cos(Theta2)
        S3 = Y - l4*np.sin(phi) - l2*np.sin(Theta2) - d
        Theta3 = np.arctan2(S3, C3)
        Theta2_list.append(Theta2)
        Theta3_list.append(Theta3)
        Xo = np.cos(Theta1)*(l2 * np.cos(Theta2) + l3 * np.cos(Theta3) + l4 * np.cos(phi))
        Yo = d + l2 * np.sin(Theta2) + l3 * np.sin(Theta3) + l4 * np.sin(phi)
        Zo = np.sin(Theta1)*(l2 * np.cos(Theta2) + l3 * np.cos(Theta3) + l4 * np.cos(phi))
        errorX = X - Xo
        errorY = Y - Yo
        errorZ = Z - Zo
        xo.append(Xo)
        yo.append(Yo)
        zo.append(Zo)
        error_net.append((errorX**2 + errorY**2 + errorZ**2)**(1/2))    
    min_index = error_net.index(min(error_net))
    error = np.round(error_net[min_index],4)
    Theta2_optimised = Theta2_list[min_index]
    Theta3_optimised = Theta3_list[min_index]
    Xe = xo[min_index]
    Ye = yo[min_index]
    Ze = zo[min_index]
    Theta = np.round([np.degrees(Theta1), np.degrees(Theta2_optimised), np.degrees(Theta3_optimised), np.degrees(phi)], 2)
    Theta2f = Theta[1]
    Theta3f = Theta[2]
    Theta4f = Theta[3]
    Theta2m = Theta2f 
    Theta3m = 90 + Theta3f - Theta2f
    Theta4m = 90 + Theta3f - Theta4f
    Theta_motors = np.round([Theta[0], Theta2m, Theta3m, Theta4m, 0])
    return Theta_motors, error

In [3]:
#time.sleep(5)
#-------------------------------------------------Main Autonomous Robot----------------------------------------------------
#-----------------------------------------------------Initiate Scan--------------------------------------------------------
Theta = np.array([0.,134.,5.,179.,0.])
Time = Theta*0.01031 + 0.544
RB = abs(Time*24e3/16)
RB_array = np.array(RB, dtype = np.single) 
RB_binary = RB_array.tobytes() 
for i in np.arange(1,6,1):
    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
    serialPort.write(voyager.data)
    time.sleep(0.1)
    readserial = (serialPort.read(size = 8))
    distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
    distance = struct.unpack('f',bytes(distarray)) 
    distance = np.array(distance)
reverse = False
while True:
    clear_output()
    print("These are the current coordinates of the robotic arm: ", Theta)
    if keyboard.is_pressed('q'):
        clear_output()
        break
    fprobability = open("C:/ODA/models/research/object_detection/Probability.txt","r")
    fclass = open("C:/ODA/models/research/object_detection/Class.txt","r")
    fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
    coordinates = fcoordinates.readlines()  
    probabilty = fprobability.read()
    classes = fclass.read()
    if (classes == "mouse\n") or (classes == "bottle\n") or (classes == "sports ball\n") or (classes == "banana\n"): # object desired has been located
        clear_output()
        time.sleep(1)
        detected = False
        if coordinates == []:
            while coordinates == []:
                fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
                coordinates = fcoordinates.readlines()  
                fcoordinates.close()
        print('Item detected: ',classes)
        print('Confidence: ',probabilty)
        #---------------------------------checking if distance of sonar works first---------------------------------------------------------
        i = 1
        Time = Theta*0.01031 + 0.544
        RB = abs(Time*24e3/16)
        RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
        RB_binary = RB_array.tobytes()
        voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
        serialPort.write(voyager.data) 
        readserial = (serialPort.read(size = 8))
        distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
        distance = struct.unpack('f',bytes(distarray)) 
        distance = np.array(distance)
        distance[0] = 0.01*(distance[0] + 11)
        print('This is the distance detected by the sonar ',distance,'m')
        #--------------------------------------------Uses monovision ------------------------------------------------------------------------------
        if (distance[0] > 0.45) or (distance[0] < 0.14):
            print("MONOVISION - Active")
            while detected == False:
                #---------------------------------This is the algebraic monovision distance formula-----------------
                from numpy import cos,sin,tan,degrees,radians
                l1 = 0.13 
                l2 = 0.13
                l3 = 0.16
                lc = 0.06 
                D = 0.07
                hc = 0.05
                #all angles are relative to ground
                T1 = 134 
                T2 = 49
                T3 = -40
                Tc = 90 - abs(T3)
                FOV = 30.5
                Xc = -l1*sin(radians(T1)) + l2*cos(radians(T2)) + lc*cos(radians(T3)) + hc*cos(radians(Tc))
                Yc = D + l1*cos(radians(180 - T1)) + l2*sin(radians(T2)) + lc*sin(radians(T3)) + hc*sin(radians(Tc))
                Xp1 = Xc - Yc/tan(radians(180 - abs(T3) - FOV/2)) 
                Xp2 = Xc + Yc/tan(radians(abs(Tc) - FOV/2)) 
                camera_length = np.round((Xp2-Xp1),2)
                fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
                coordinates = fcoordinates.readlines()  
                if coordinates == []:
                    while coordinates == []:
                        fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
                        coordinates = fcoordinates.readlines()  
                        fcoordinates.close()
                bCoordinates = True
                i = 0
                value = []
                for k in coordinates:
                    i = i + 1
                    if (i == 1) or (i == 3):
                        value.append(np.round((float(k)),2))
                    else:
                        value.append(np.round((float(k)),2))           
                yminr = value[0]
                xminr = value[1]
                ymaxr = value[2]
                xmaxr = value[3]
                fcoordinates.close()
                distance = [np.round((1-ymaxr) * camera_length + Xp1, 4)]   
                if distance[0] < 0.45:
                    detected = True
        else:
            print('SONAR - Active')
        print('\nDistance to object: ',distance[0],'m')   
        #-----------------------------------Centerring the object is the first step-------------------------------------       
        i = 0
        value = []
        for k in coordinates:
            i = i + 1
            if (i == 1) or (i == 3):
                value.append(np.round((float(k) * 480),2))
            else:
                value.append(np.round((float(k) * 640),2))         
        ymin = value[0]
        xmin = value[1]
        ymax = value[2]
        xmax = value[3]
        width = xmax - xmin
        pixelmovement = (320 + width*0.5) - xmax
        pixelratio = abs(pixelmovement)/320
        R1 = np.tan(np.radians(20.5)) * (distance[0])
        realmovement = R1*pixelratio
        rotation = np.degrees(np.arctan(realmovement/distance[0])) 
        print('Rotation = ',rotation)
        #-----------------------------------base Rotation to center-----------------------------------
        n = 10
        base_increment = rotation/n
        if pixelmovement > 0: # left motion - Anticlockwise rotation
            for p in np.arange(0,n,1):
                Theta[0] = Theta[0] - base_increment
                Time = Theta*0.01031 + 0.544
                RB = abs(Time*24e3/16)
                RB_array = np.array(RB, dtype = np.single) 
                RB_binary = RB_array.tobytes() 
                i = 1 
                voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
                serialPort.write(voyager.data)   
                time.sleep(0.05)
        elif pixelmovement < 0: # right motion - Clockwise rotation
            for p in np.arange(0,n,1):
                Theta[0] = Theta[0] + base_increment 
                Time = Theta*0.01031 + 0.544
                RB = abs(Time*24e3/16)
                RB_array = np.array(RB, dtype = np.single) 
                RB_binary = RB_array.tobytes() 
                i = 1 
                voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
                serialPort.write(voyager.data) 
                time.sleep(0.05)  
        if Theta[0] > 90:
            Xe = distance[0] * np.cos(np.radians(Theta[0]))
            Ze = distance[0] * np.sin(np.radians(Theta[0]))
        elif Theta[0] < 90:
            Xe = - (distance[0] * np.cos(np.radians(180 - Theta[0])))
            Ze = distance[0] * np.sin(np.radians(180 - Theta[0]))
        elif Theta[0] == 90:
            Xe = 0
            Ze = distance[0]
        if (classes == "mouse\n"): # Height of subject must change based on object
            Ye = -0.02
        else:
            Ye = 0.01
        Xe = np.round(Xe,2)
        Ze = abs(np.round(Ze,2))
        print("\nCoordinates Detected \nX:",Xe, "\nY:",Ye, "\nZ:",Ze)        
        #-------------------------------------------Inverse Kinmatics 1--------------------------------------------------
        e_list = []
        T1 = []
        T2 = []
        T3 = []
        T4 = []
        T5 = []
        xi = []
        phioptimization = np.arange(-90,91,1)
        for phi in phioptimization:
            if Xe == 0:
                Tim,ei = Inverse_Kinematics(Ze,Ye,Xe,phi)
                Tim[0] = 90
            else:
                Tim,ei = Inverse_Kinematics(Xe,Ye,Ze,phi)
            Possible = True
            for k in np.arange(0,len(Tim),1):
                if (Tim[k] < 0) or (Tim[k] > 180):
                    Possible = False
            if Possible:
                e_list.append(ei)
                T1.append(Tim[0])
                T2.append(Tim[1])
                T3.append(Tim[2])
                T4.append(Tim[3])
                T5.append(Tim[4])
                xi.append(phi)
                if ei < 0.001:
                    break   
        if e_list == []:
            print('This cooridnate cannot be reached')
            sys.exit()
        else:
            min_error_index = e_list.index(min(e_list))
            error_motor = np.round(e_list[min_error_index],4)
            phi_final = xi[min_error_index]
            Theta_final_choice = np.round([T1[min_error_index], T2[min_error_index], T3[min_error_index], T4[min_error_index], 0])
            route = True
            Theta = np.round(Theta)
            while route:
                if Theta[0] != Theta_final_choice[0]:
                    if (Theta_final_choice[0] - Theta[0]) > 0:
                        Theta[0] = Theta[0] + 0.5
                    else:
                        Theta[0] = Theta[0] - 0.5
                if Theta[1] != Theta_final_choice[1]:
                    if (Theta_final_choice[1] - Theta[1]) > 0:
                        Theta[1] = Theta[1] + 0.5
                    else:
                        Theta[1] = Theta[1] - 0.5
                if Theta[2] != Theta_final_choice[2]:
                    if (Theta_final_choice[2] - Theta[2]) > 0:
                        Theta[2] = Theta[2] + 0.5
                    else:
                        Theta[2] = Theta[2] - 0.5
                if Theta[3] != Theta_final_choice[3]:
                    if (Theta_final_choice[3] - Theta[3]) > 0:
                        Theta[3] = Theta[3] + 0.5
                    else:
                        Theta[3] = Theta[3] - 0.5
                print('Stuck in Kinematics 1')   
                print('Theta in kinematics 1',Theta)
                print('Where Theta should be:',Theta_final_choice)
                if (Theta[0] == Theta_final_choice[0]) and (Theta[1] == Theta_final_choice[1]) and (Theta[2] == Theta_final_choice[2]) and (Theta[3] == Theta_final_choice[3]):
                    route = False
                    print('Kinematics 1 route success')   
                Time = Theta*0.01031 + 0.544
                RB = abs(Time*24e3/16)
                RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
                RB_binary = RB_array.tobytes()
                for i in np.arange(1,6,1):
                    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
                    serialPort.write(voyager.data)
                    readserial = (serialPort.read(size = 8))
                    distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
                    distance = struct.unpack('f',bytes(distarray))
                time.sleep(0.01)                       
        print('\nFor coordinates X = ',Xe, 'Y = ',Ye, 'Z = ',Ze)
        print('Theta after first inverse kinematics = ',Theta)
        #------------------------------------time stop and then close gripper to grab object-----------------------------------       
        time.sleep(0.5)        
        i = 5
        Theta[4] = 90
        Time = Theta*0.01031 + 0.544
        RB = abs(Time*24e3/16)
        RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
        RB_binary = RB_array.tobytes()
        voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
        serialPort.write(voyager.data)
        #------------This is going to bring the arm up so that object does not scrape against the ground----------------
        n = 100
        for k in np.arange(n):
            i = 2 
            Theta[1] = Theta[1] + 0.5
            Time = Theta*0.01031 + 0.544
            RB = abs(Time*24e3/16)
            RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
            RB_binary = RB_array.tobytes()
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            serialPort.write(voyager.data)
            readserial = (serialPort.read(size = 8))
            distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
            distance = struct.unpack('f',bytes(distarray))
            time.sleep(0.01)          
        #-------------------------------------------end point placement ------------------------------------------------------
        if (classes == "mouse\n"):
            Xe = -0.30
            Ye = 0.17
            Ze = 0.17
        elif (classes == "bottle\n"):
            Xe = -0.30
            Ye = 0.15
            Ze = 0.30  
        elif (classes == "sports ball\n"):
            Xe = -0.3
            Ye = 0.17
            Ze = 0
        elif (classes =='banana\n'):
            Xe = -0.3
            Ye = 0.2
            Ze = 0
        # --------------------------------------------Inverse Kinematics 2----------------------------------------------------
        e_list = []
        T1 = []
        T2 = []
        T3 = []
        T4 = []
        T5 = []
        xi = []
        phioptimization = np.arange(-90,91,1)
        for phi in phioptimization:
            if Xe == 0:
                Tim,ei = Inverse_Kinematics(Ze,Ye,Xe,phi)
                Tim[0] = 90
            else:
                Tim,ei = Inverse_Kinematics(Xe,Ye,Ze,phi)
            Possible = True
            for k in np.arange(0,len(Tim),1):
                if (Tim[k] < 0) or (Tim[k] > 180):
                    Possible = False
            if Possible:
                e_list.append(ei)
                T1.append(Tim[0])
                T2.append(Tim[1])
                T3.append(Tim[2])
                T4.append(Tim[3])
                T5.append(Tim[4])
                xi.append(phi)
                if ei < 0.001:
                    break         
        if e_list == []:
            print('This cooridnate cannot be reached')
            sys.exit()
        else:
            min_error_index = e_list.index(min(e_list))
            error_motor = np.round(e_list[min_error_index],4)
            phi_final = xi[min_error_index]
            Theta_final_choice = np.round([T1[min_error_index], T2[min_error_index], T3[min_error_index], T4[min_error_index], 90])
            print('\nFor coordinates X = ',Xe, 'Y = ',Ye, 'Z = ',Ze)
            print('Theta the robot must go to after second inverse kinematics = ',Theta_final_choice)
            print('The current Theta values = ',Theta,'before kinematics 2\n')
            route = True
            Theta = np.round(Theta)
            while route:
                if Theta[0] != Theta_final_choice[0]:
                    if (Theta_final_choice[0] - Theta[0]) > 0:
                        Theta[0] = Theta[0] + 0.5
                    else:
                        Theta[0] = Theta[0] - 0.5
                if Theta[1] != Theta_final_choice[1]:
                    if (Theta_final_choice[1] - Theta[1]) > 0:
                        Theta[1] = Theta[1] + 0.5
                    else:
                        Theta[1] = Theta[1] - 0.5
                if Theta[2] != Theta_final_choice[2]:
                    if (Theta_final_choice[2] - Theta[2]) > 0:
                        Theta[2] = Theta[2] + 0.5
                    else:
                        Theta[2] = Theta[2] - 0.5
                if Theta[3] != Theta_final_choice[3]:
                    if (Theta_final_choice[3] - Theta[3]) > 0:
                        Theta[3] = Theta[3] + 0.5
                    else:
                        Theta[3] = Theta[3] - 0.5
                Theta[4] = 90
                print('\nTheta inside Kinematics 2',Theta)
                print('What Theta should be ',Theta_final_choice)
                if (Theta[0] == Theta_final_choice[0]) and (Theta[1] == Theta_final_choice[1]) and (Theta[2] == Theta_final_choice[2]) and (Theta[3] == Theta_final_choice[3]):
                    route = False
                    print('Kinematics 2 route success')
                Time = Theta*0.01031 + 0.544
                RB = abs(Time*24e3/16)
                RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
                RB_binary = RB_array.tobytes()
                for i in np.arange(1,6,1):
                    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
                    serialPort.write(voyager.data)
                    readserial = (serialPort.read(size = 8))
                    distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
                    distance = struct.unpack('f',bytes(distarray))
                time.sleep(0.01)
        # -----------------------------------------------releasing object--------------------------------------------------
        time.sleep(1)
        i = 5
        Theta[4] = 0
        Time = Theta*0.01031 + 0.544
        RB = abs(Time*24e3/16)
        RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
        RB_binary = RB_array.tobytes()
        voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
        serialPort.write(voyager.data)  
        readserial = (serialPort.read(size = 8))
        distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
        distance = struct.unpack('f',bytes(distarray))
        time.sleep(1)
        n = 100
        i = 2 
        for k in np.arange(n):
            Theta[1] = Theta[1] + 0.5
            Time = Theta*0.01031 + 0.544
            RB = abs(Time*24e3/16)
            RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
            RB_binary = RB_array.tobytes()
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            serialPort.write(voyager.data)
            readserial = (serialPort.read(size = 8))
            distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
            distance = struct.unpack('f',bytes(distarray))
            time.sleep(0.01)        
        # -----------------------------------back to origin for scanning to take place again-----------------------------------
        Theta_scan = np.array([0,134,5,179,0])
        route = True
        Theta = np.round(Theta)
        while route:
            if Theta[0] != Theta_scan[0]:
                Theta[0] = Theta[0] - 0.5
            if Theta[1] != Theta_scan[1]:
                if (Theta_scan[1] - Theta[1]) > 0:
                    Theta[1] = Theta[1] + 0.5
                else:
                    Theta[1] = Theta[1] - 0.5
            if Theta[2] != Theta_scan[2]:
                if (Theta_scan[2] - Theta[2]) > 0:
                    Theta[2] = Theta[2] + 0.5
                else:
                    Theta[2] = Theta[2] - 0.5
            if Theta[3] != Theta_scan[3]:
                if (Theta_scan[3] - Theta[3]) > 0:
                    Theta[3] = Theta[3] + 0.5
                else:
                    Theta[3] = Theta[3] - 0.5
            Theta[4] = 0
            if (Theta[0] == Theta_scan[0]) and (Theta[1] == Theta_scan[1]) and(Theta[2] == Theta_scan[2]) and (Theta[3] == Theta_scan[3]):
                route = False
                print('Home route success')
            Time = Theta*0.01031 + 0.544
            RB = abs(Time*24e3/16)
            RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
            RB_binary = RB_array.tobytes()
            for i in np.arange(1,6,1):
                voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
                serialPort.write(voyager.data)  
                readserial = (serialPort.read(size = 8))
                distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
                distance = struct.unpack('f',bytes(distarray))
            time.sleep(0.01)
        reverse = False
        distance = np.array(distance)
        distance[0] = 0
        fclass.close()
        fprobability.close()
        fclass = open("C:/ODA/models/research/object_detection/Class.txt", "w+")
        fclass.write("{}\n".format('NONE'))
        fclass.close()
    #--------------------------------------------------initialize scan---------------------------------------------------
    if (Theta[0] < 180) and (reverse == False):
        Theta[0] = Theta[0] + 0.5 
        print('Theta:',Theta)
        for i in np.arange(1,6,1):
            Time = Theta*0.01031 + 0.544
            RB = abs(Time*24e3/16)
            RB_array = np.array(RB, dtype = np.single) 
            RB_binary = RB_array.tobytes()
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            serialPort.write(voyager.data) 
            readserial = (serialPort.read(size = 8))
            distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
            distance = struct.unpack('f',bytes(distarray)) 
        time.sleep(0.02) 
        if Theta[0] >= 180:
            reverse = True
            print("Search begins in opposite direction")
    else:
        Theta[0] = Theta[0] - 0.5
        print('Theta:',Theta)
        for i in np.arange(1,6,1):
            Time = Theta*0.01031 + 0.544
            RB = abs(Time*24e3/16)
            RB_array = np.array(RB, dtype = np.single) 
            RB_binary = RB_array.tobytes()
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            serialPort.write(voyager.data) 
            readserial = (serialPort.read(size = 8))
            distarray = [readserial[2],readserial[3],readserial[4],readserial[5]]
            distance = struct.unpack('f',bytes(distarray)) 
        time.sleep(0.02) 
        if Theta[0] <= 0:
            reverse = False
            print("Search begins in forward direction")
    fclass.close()
    fprobability.close()
    fcoordinates.close()
print('Voyager Offline')

Voyager Offline
